In [1]:
include("../src/RadialBasisFunction.jl")
include("../src/DiscreteAdjoint.jl")
include("../src/GradientDescentTools.jl")

using LinearAlgebra
using .RadialBasisFunction
using .DiscreteAdjoint
using .GradientDescentTools

## Adjoint method applied to RBF approximation

Fixed some paramters, we obtain an approximated field solving the steady-state heat equation

In [2]:
a, b = 0, 2π
N = 40
X = range(a,b, length=N)
boundary_idx = [1, N]
X_boundary = X[boundary_idx]
internal_idx = 2:(N-1)
X_internal = X[internal_idx]


u(x) = sin.(x)   # Not known in advance
q(x) = -sin.(x)  # Known in advance (param)

boundary_conditions = u(X_boundary)


φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)


n_points_in_stencil = 7


C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idx, boundary_idx, boundary_conditions, q)

38-element Vector{Float64}:
  0.16230630440607596
  0.3184441606794323
  0.46639819882924805
  0.6024453236002578
  0.7229552007955176
  0.8247685909891668
  0.9052437503888441
  0.9622961535265828
  0.9944469507504687
  1.000861661907953
  ⋮
 -0.9944469507504649
 -0.96229615352658
 -0.9052437503888425
 -0.8247685909891656
 -0.7229552007955165
 -0.602445323600257
 -0.46639819882924743
 -0.31844416067943265
 -0.16230630440607627

Now we want to find a specific field, $u_{opt}$, that satisfy some properties defined by $J(u)$. $J$ can be thought as a "quality" function if it has to be maximized or as a "cost" if it has to be minimized.  
To reach our goal we can modify some parameters of the problem. In this case we can modify the value of the function $q$ at each point of the object (so we can decide its shape: thus $q$ itself)  

In [3]:
# Note that the optimization is carried out only on internal points (in fact boundary poiints are conditioned)...
C_internal = C[internal_idx, internal_idx]

# Defininig J (obj function) as follows menas ~> We want to reach a designed u_target
u_target(x) = (x .- a) .* (b .- x)
u_target_num = u_target(X_internal)
J(u_num) = norm(u_num - u_target_num)^2

g(u_num) = 2 * (u_num - u_target_num)

# Define ideal parameters that leads to u_target
h_target_num = C_internal * u_target_num 
q_target_num = h_target_num + C[internal_idx, boundary_idx]*boundary_conditions
error_h(h_num) = norm(h_num-h_target_num)^2

error_h (generic function with 1 method)

Getting the sougth parameters $q$ that leads to the desired field $u_{opt}$

In [4]:
h_0 = C_internal * u_approximated
N_opt = 1e4

h_opt_num, u_opt_num = adjoint_opt_BB(C_internal, h_0, J, g, N_opt, BB_long_step)
q_opt_num = h_opt_num + + C[internal_idx, boundary_idx]*boundary_conditions

38-element Vector{Float64}:
 -1.9999999246579396
 -2.000000155206049
 -1.9999997815552175
 -2.000000267900302
 -1.9999997014376807
 -2.0000003096530254
 -1.9999996974140248
 -2.000000280039484
 -1.9999997539957215
 -2.0000002048682024
  ⋮
 -1.9999997768071243
 -2.0000002556316576
 -1.9999997225147914
 -2.000000284970659
 -1.999999724539428
 -2.0000002476554997
 -1.9999997977895774
 -2.0000001438195083
 -1.9999999301273892

## Performances

In [5]:
relative_error_on_q = norm(q_opt_num - q_target_num) / norm(q_target_num)
relative_error_on_u = norm(u_opt_num - u_target_num) / norm(u_target_num)

println("Relative error on u: $(relative_error_on_u)")
println("Relative error on q: $(relative_error_on_q)")

Relative error on u: 5.6570266610036386e-11
Relative error on q: 8.903298821272255e-8


In [7]:
h0 = q(X[internal_idx]) - C[internal_idx, boundary_idx]*boundary_conditions
N_iter = 1e4

h_opt, u_opt = adjoint_opt_BB(C[internal_idx, internal_idx], h0, J, g, N_iter, BB_long_step, 10^(-14))
q_opt = h_opt + C[internal_idx, boundary_idx]*boundary_conditions
println("Fitness value is: $(J(u_opt))")
println("Errors on the obtained q: $(q_opt - q_target_num)")
println("Errors on the obtained u: $(u_opt - u_target_num)")
println("Relative error on u: $(norm(u_opt - u_target_num) / norm(u_target_num))")
println("Relative error on q: $(norm(q_opt - q_target_num) / norm(q_target_num))")

Fitness value is: 1.691663124421444e-16


Errors on the obtained q: [-4.111116025384831e-7, 8.430995495078974e-7, -1.1798254431205635e-6, 1.432796862488317e-6, -1.5752640529775874e-6, 1.6072680055767563e-6, -1.5413140381781432e-6, 1.3957606352921204e-6, -1.194570854234911e-6, 9.633345203763355e-7, -7.274159696812887e-7, 5.072434521302682e-7, -3.185547843287395e-7, 1.7012809649941119e-7, -6.520061157644363e-8, 1.7392232098956129e-9, 2.6133885056012218e-8, -2.672942533976652e-8, 1.0074708178819947e-8, 1.3243232732662591e-8, -3.233291323567755e-8, 3.6537456171714666e-8, -1.6028207650009563e-8, -3.769955059240715e-8, 1.3060312564761034e-7, -2.649781185581901e-7, 4.3854503339879614e-7, -6.427947027987102e-7, 8.639953315014992e-7, -1.0819910909454222e-6, 1.273878700747133e-6, -1.4143536568589354e-6, 1.4812803654784545e-6, -1.456427245472014e-6, 1.3279473867200409e-6, -1.0954334874746507e-6, 7.836365472080331e-7, -3.8250568623610093e-7]
Errors on the obtained u: [9.935791078774514e-10, -1.954612693921831e-9, 2.8036675203679806e-9, -3